In [3]:
from scipy.stats import entropy
import nibabel as nib 
import numpy as np 
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
import torch

In [4]:
#check which statistic is most useful to train model with segmentation masks
# statistics include: mean, standard deviation, variance, skewnee, kurtosis, entropy

#load in the data
hard_image = '/media/krishar1/Elements1/AnatomyConstrainedMultipathGAN/B30f_B50f/hard_masked/118866/118866.nii.gz'
soft_image = '/media/krishar1/Elements1/AnatomyConstrainedMultipathGAN/B30f_B50f/soft_masked/118866/118866.nii.gz'
hard_img = nib.load(hard_image)
soft_img = nib.load(soft_image)
hard_data = hard_img.get_fdata()
soft_data = soft_img.get_fdata()

#Clip and normalize the image 
clipped_hard = np.clip(hard_data, -1024, 3072)
clipped_soft = np.clip(soft_data, -1024, 3072)
normalizer = interp1d([-1024,3072], [-1,1])
norm_data_hard = normalizer(clipped_hard)
norm_data_soft = normalizer(clipped_soft)

#calculate the statistics
hard_mean = np.mean(norm_data_hard)
soft_mean = np.mean(norm_data_soft)
hard_mean, soft_mean

(-0.78437651960053, -0.7854122403153297)

In [ ]:
hard_slice = nib.load("/home-local/Kernel_Conversion/MultipathKernelConversion_forA6000/multipath_data_journalextension/train_ge_bone_hard/119740_098.nii.gz")
soft_slice = nib.load("/home-local/Kernel_Conversion/MultipathKernelConversion_forA6000/multipath_data_journalextension/train_ge_bone_soft/119740_098.nii.gz")
hard_mask_slice = nib.load("/fs5/p_masi/krishar1/MIDL/STANDARD_BONE/hard_slices/119740_098.nii.gz")
soft_mask_slice = nib.load("/fs5/p_masi/krishar1/MIDL/STANDARD_BONE/soft_slices/119740_098.nii.gz")

hard_slice_data = hard_slice.get_fdata()[:,:,0]
soft_slice_data = soft_slice.get_fdata()[:,:,0]
hard_mask_slice_data = hard_mask_slice.get_fdata()[:,:,0]
soft_mask_slice_data = soft_mask_slice.get_fdata()[:,:,0]

normalizer = interp1d([-1024,3072], [-1,1])
clipped_hard = np.clip(hard_slice_data, -1024, 3072)
clipped_soft = np.clip(soft_slice_data, -1024, 3072)
norm_data_hard = normalizer(clipped_hard)
norm_data_soft = normalizer(clipped_soft)

#Check the unique values in the mask
np.unique(hard_mask_slice_data), np.unique(soft_mask_slice_data)

In [7]:
#For every label in the mask, calculate the mean for the hard and soft image 
hard_means = []
soft_means = []
hard_std = []
soft_std = []
for label in np.unique(hard_mask_slice_data):
    if label == 0:
        continue
    hard_means.append(np.mean(norm_data_hard[hard_mask_slice_data == label]))
    soft_means.append(np.mean(norm_data_soft[soft_mask_slice_data == label]))
    hard_std.append(np.std(norm_data_hard[hard_mask_slice_data == label]))
    soft_std.append(np.std(norm_data_soft[soft_mask_slice_data == label]))

#Sum hard and soft means
hard_sum_mean = np.sum(hard_means)
soft_sum_mean = np.sum(soft_means)

hard_sum_std = np.sum(hard_std)
soft_sum_std = np.sum(soft_std)
hard_sum_mean, soft_sum_mean, hard_sum_std, soft_sum_std

(-6.190911822998083,
 -6.202749545012664,
 0.9991052214759946,
 0.6840271616155018)

In [2]:
#Convert numpy array to a pytroch tensor of size (1,1,512,512)
hard_tensor = torch.tensor(norm_data_hard).unsqueeze(0).unsqueeze(0)
soft_tensor = torch.tensor(norm_data_soft).unsqueeze(0).unsqueeze(0)
hard_mask_tensor = torch.tensor(hard_mask_slice_data).unsqueeze(0).unsqueeze(0)
soft_mask_tensor = torch.tensor(soft_mask_slice_data).unsqueeze(0).unsqueeze(0)

#Masks have multiple label, compute mean for each label
hard_means = []
soft_means = []
hard_std = []
soft_std = []
for label in np.unique(hard_mask_tensor):
    if label == 0:
        continue
    hard_means.append(torch.mean(hard_tensor[hard_mask_tensor == label]))
    soft_means.append(torch.mean(soft_tensor[soft_mask_tensor == label]))
    hard_std.append(torch.std(hard_tensor[hard_mask_tensor == label]))
    soft_std.append(torch.std(soft_tensor[soft_mask_tensor == label]))



NameError: name 'norm_data_hard' is not defined

In [13]:
hard_sum_mean = torch.sum(torch.tensor(hard_means))
soft_sum_mean = torch.sum(torch.tensor(soft_means))
hard_sum_std = torch.sum(torch.tensor(hard_std))
soft_sum_std = torch.sum(torch.tensor(soft_std))

hard_sum_mean, soft_sum_mean, hard_sum_std, soft_sum_std

(tensor(-6.1909, dtype=torch.float64),
 tensor(-6.2027, dtype=torch.float64),
 tensor(0.9997, dtype=torch.float64),
 tensor(0.6844, dtype=torch.float64))

In [14]:
torch.unique(hard_mask_tensor), torch.unique(soft_mask_tensor)

(tensor([ 0.,  1.,  2.,  3.,  4.,  8., 10., 13., 15., 16., 18., 25., 26.],
        dtype=torch.float64),
 tensor([ 0.,  1.,  2.,  3.,  4.,  8., 10., 13., 15., 16., 18., 25., 26.],
        dtype=torch.float64))

In [16]:
np.unique(hard_mask_tensor), np.unique(soft_mask_tensor)

(array([ 0.,  1.,  2.,  3.,  4.,  8., 10., 13., 15., 16., 18., 25., 26.]),
 array([ 0.,  1.,  2.,  3.,  4.,  8., 10., 13., 15., 16., 18., 25., 26.]))

In [17]:
hard_mask_tensor

tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]], dtype=torch.float64)